# Macro Indicators: Trend and Momentum Baseline

This notebook demonstrates a clear macro-series workflow for community research.

## What this notebook teaches
- Centralized FinUties API auth from `notebooks/.env`
- Schema normalization for time series endpoints
- EWMA trend + rate-of-change overlay interpretation

In [ ]:
from pathlib import Path
import os

import matplotlib.pyplot as plt
import pandas as pd
import requests
from dotenv import load_dotenv

load_dotenv(Path.cwd().parent / ".env")
API_ORIGIN = os.getenv("FINUTIES_API_ORIGIN", "https://data.finuties.com").rstrip("/")
API_KEY = os.getenv("FINUTIES_API_KEY", "").strip()
if not API_KEY:
    raise ValueError("Missing FINUTIES_API_KEY in notebooks/.env")
HEADERS = {"Authorization": f"Bearer {API_KEY}"}

In [ ]:
def finuties_rows(endpoint: str, params: dict | None = None) -> list[dict]:
    response = requests.get(
        f"{API_ORIGIN}{endpoint}",
        headers=HEADERS,
        params=params or {},
        timeout=30,
    )
    response.raise_for_status()
    payload = response.json()
    if isinstance(payload, list):
        return payload
    return payload.get("items") or payload.get("data") or []

rows = finuties_rows("/api/v1/data/macro/series", {"limit": 300})
df = pd.DataFrame(rows)
if not {"date", "value"}.issubset(df.columns):
    raise ValueError(f"Unexpected macro schema: {list(df.columns)}")

In [ ]:
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["value"] = pd.to_numeric(df["value"], errors="coerce")
df = df.dropna(subset=["date", "value"]).sort_values("date").reset_index(drop=True)

df["ewma_20"] = df["value"].ewm(span=20, adjust=False).mean()
df["roc_12"] = df["value"].pct_change(12) * 100

df[["date", "value", "ewma_20", "roc_12"]].tail(10)


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].plot(df["date"], df["value"], label="Value", linewidth=1.7, color="#1f77b4")
axes[0].plot(df["date"], df["ewma_20"], label="EWMA(20)", linewidth=1.7, color="#ff7f0e")
axes[0].set_title("Macro Level and Trend")
axes[0].set_xlabel("Date")
axes[0].set_ylabel("Value (units)")
axes[0].legend()

axes[1].plot(df["date"], df["roc_12"], label="12-period ROC", linewidth=1.7, color="#2ca02c")
axes[1].axhline(0, color="#444444", linewidth=1, linestyle="--")
axes[1].set_title("Momentum Context")
axes[1].set_xlabel("Date")
axes[1].set_ylabel("Percent (%)")
axes[1].legend()

plt.tight_layout()
plt.show()

## Interpretation and caveats

- EWMA smooths noise and highlights direction, but lags turning points.
- ROC helps detect acceleration or deceleration phases.
- Validate anomalies against release calendars and revisions.

## Community extension ideas

- Compare multiple macro series on a normalized z-score basis.
- Add recession/expansion shading using your own event set.